In [2]:
import pandas as pd
import numpy as np
from __future__ import division

In [3]:
matchInfo = pd.read_csv("Datasets/matches.csv")
ballInfo = pd.read_csv("Datasets/deliveries.csv")

In [8]:
# Batsman inconsistency 
# Inconsistency is the standard deviation of all individual scores (both outs and not out scores)
# Player should score atleast 1000 runs and played atleast 15  matches
batsmanRuns = ballInfo[['batsman','match_id','batsman_runs']]
batsmanRuns = batsmanRuns.groupby(['batsman','match_id'],as_index=False)['batsman_runs'].sum()
matchesPlayed = batsmanRuns.groupby(['batsman'],as_index=False)['match_id'].count()
matchesPlayed = matchesPlayed.rename(index=str, columns={"match_id": "matches_played"})
filteredPlayers = matchesPlayed[matchesPlayed['matches_played'] > 1]
filteredPlayers = filteredPlayers['batsman']
batsmanRuns = batsmanRuns.loc[batsmanRuns['batsman'].isin(filteredPlayers)]
batsmanRuns = batsmanRuns[['batsman','batsman_runs']]
batsmanStd = batsmanRuns.groupby(['batsman'])['batsman_runs'].std()
batsmanStd = batsmanStd.reset_index()
batsmanStd = batsmanStd.rename(index=str, columns={"batsman_runs": "inconsistency (std)"})

batsmanRuns = ballInfo.groupby(['batsman'],as_index=False).agg({"batsman_runs": sum, "match_id": lambda x: x.nunique()})
batsmanRuns = batsmanRuns.rename(index=str, columns={"batsman_runs": "total_runs", "match_id": "matches_played"})
dismissalInfo = ballInfo['player_dismissed']
dismissalInfo = dismissalInfo.dropna(axis=0, how='any')
batsmanDismissal = pd.DataFrame({"batsman": dismissalInfo})
batsmanDismissal["dismissed"] = batsmanDismissal.groupby('batsman')['batsman'].transform("count")
batsmanDismissal = batsmanDismissal.drop_duplicates(subset='batsman',keep='last')
batsmanStats = pd.merge(batsmanRuns, batsmanDismissal, on=['batsman'])
batsmanStats["avg"] = batsmanStats.apply(lambda x: x["total_runs"]/x["dismissed"],axis=1)

batsmanConsistency = pd.merge(batsmanStd, batsmanStats, on=['batsman'])
batsmanConsistency = batsmanConsistency.sort_values(['inconsistency (std)','matches_played'], ascending=[0,0])
batsmanConsistency = batsmanConsistency[batsmanConsistency['matches_played'] > 15]
batsmanConsistency = batsmanConsistency[batsmanConsistency['total_runs'] > 1000]

In [9]:
batsmanConsistency.sort_values('inconsistency (std)', ascending=True)

,batsman,inconsistency (std),total_runs,matches_played,dismissed,avg
262,RA Jadeja,12.026445,1574,98,68,23.147059
124,IK Pathan,12.595426,1148,81,52,22.076923
85,DJ Bravo,15.300216,1262,81,55,22.945455
320,SS Tiwary,16.296362,1224,63,44,27.818182
180,LRPL Taylor,16.729536,1017,54,40,25.425000
239,PA Patel,17.215130,1927,100,91,21.175824
206,MK Tiwary,17.547192,1324,66,46,28.782609
215,MS Dhoni,17.905640,3270,128,83,39.397590
369,Yuvraj Singh,17.986371,2339,105,92,25.423913
154,KA Pollard,18.033056,1959,96,68,28.808824


In [11]:
# Highest contribution(percent of runs) in partnership

strikerRuns = ballInfo.groupby(['batsman'],as_index=False)['total_runs'].sum()
nonStrikerRuns = ballInfo.groupby(['non_striker'],as_index=False)['total_runs'].sum()
strikerRuns = strikerRuns.rename(index=str, columns={"batsman": "player"})
nonStrikerRuns = nonStrikerRuns.rename(index=str, columns={"non_striker": "player"})
partnerShip = pd.merge(strikerRuns,nonStrikerRuns,on=['player'])
partnerShip["total_runs"] = partnerShip.apply(lambda x: x['total_runs_x'] + x['total_runs_y'],axis=1)
partnerShip = partnerShip.drop(['total_runs_x','total_runs_y'],axis=1)
batsmanRuns = ballInfo.groupby(['batsman'],as_index=False).agg({"batsman_runs": sum})
batsmanRuns = batsmanRuns.rename(index=str, columns={"batsman": "player"})
partnerShip = pd.merge(partnerShip,batsmanRuns,on=['player'])
partnerShip["percent"] = partnerShip.apply(lambda x: x['batsman_runs']*100/x['total_runs'],axis=1)

partnerShip = partnerShip.sort_values("total_runs",ascending=False)

In [12]:
partnerShip.head(n=20)

,player,total_runs,batsman_runs,percent
402,V Kohli,8729,4115,47.141712
349,SK Raina,8349,4106,49.179542
124,G Gambhir,7971,3634,45.590265
300,RG Sharma,7789,3874,49.736808
322,S Dhawan,6936,3082,44.434833
317,RV Uthappa,6772,3390,50.059067
240,MS Dhoni,6473,3270,50.517534
20,AB de Villiers,6468,3270,50.556586
92,DA Warner,6274,3373,53.761556
75,CH Gayle,6246,3451,55.251361


In [15]:
# Highest contribution to team (Total team runs / Player runs)
# Player should play atleast 15 matches

teamRuns = ballInfo.groupby(['match_id','inning'], as_index=False).agg({"total_runs": sum})
playerRuns = ballInfo.groupby(['match_id','inning','batsman'],as_index=False).agg({"batsman_runs": sum})
playerContribution = pd.merge(teamRuns,playerRuns,on=['match_id','inning'])
playerContribution["contribution(%)"] = playerContribution.apply(lambda x: x["batsman_runs"]*100/x["total_runs"],axis=1)
playerContribution = playerContribution.groupby(['batsman'],as_index=False).agg({"contribution(%)": np.mean, "batsman_runs": sum, "match_id": lambda x: x.nunique()})
playerContribution = playerContribution.rename(index=str, columns={"match_id": "matches_played"})
playerContribution = playerContribution.sort_values('contribution(%)',ascending=False)

playerContribution = playerContribution[playerContribution['matches_played'] > 15]

In [16]:
playerContribution.head(n=20)

,batsman,batsman_runs,matches_played,contribution(%)
204,LMP Simmons,942,22,25.310098
77,CH Gayle,3451,91,24.981805
352,SE Marsh,2225,61,23.208250
94,DA Warner,3373,100,21.209335
227,MEK Hussey,1977,58,20.725565
236,ML Hayden,1107,32,20.197821
409,V Kohli,4115,131,20.181374
35,AM Rahane,2675,89,19.394162
368,SR Tendulkar,2334,78,19.241636
367,SPD Smith,1241,47,19.215368


In [5]:
# Highest consecutive number of matches taking atleast 1 wicket

notbowlerDismissals = ['run out','hit wicket','obstructing the field','hit wicket','retired hurt']
dismissalInfo = ballInfo[['match_id','bowler','dismissal_kind']]
dismissalInfo = dismissalInfo[~dismissalInfo['dismissal_kind'].isin(notbowlerDismissals)]
dismissalInfo = dismissalInfo.fillna(0)
dismissalInfo['dismissal_kind'] = dismissalInfo['dismissal_kind'].replace(['caught', 'bowled', 'lbw', 'stumped','caught and bowled'], 1)
consistentBowler = dismissalInfo.groupby(['bowler','match_id'],as_index=False)['dismissal_kind'].sum()
consistentBowler = consistentBowler.sort_values(['bowler','match_id'], ascending=[1,1])

consistency = pd.DataFrame(columns=['bowler', 'consec_matches'])
playerMax = 0
currMax = 0
startIdx = 0
for i in range(0, len(consistentBowler)):
    if i == 0:
        if consistentBowler.loc[i,'dismissal_kind'] > 0:
            currMax = currMax + 1
    elif consistentBowler.loc[i,'bowler'] == consistentBowler.loc[i-1,'bowler']:
        if consistentBowler.loc[i,'dismissal_kind'] == 0:
            if playerMax < currMax:
                playerMax = currMax
            currMax = 0;
        else:
            currMax = currMax + 1
    else:
        consistency.loc[startIdx,"bowler"] = consistentBowler.loc[i-1, 'bowler'];
        consistency.loc[startIdx,"consec_matches"] = playerMax
        startIdx = startIdx + 1
        playerMax = 0
        currMax = 0

notbowlerDismissals = ['run out','hit wicket','obstructing the field','hit wicket','retired hurt']
dismissalInfo = ballInfo[['match_id','bowler','dismissal_kind']]
dismissalInfo = dismissalInfo[~dismissalInfo['dismissal_kind'].isin(notbowlerDismissals)]
dismissalInfo = dismissalInfo.fillna(0)
dismissalInfo['dismissal_kind'] = dismissalInfo['dismissal_kind'].replace(['caught', 'bowled', 'lbw', 'stumped','caught and bowled'], 1)
dismissalInfo = dismissalInfo.groupby(['bowler'],as_index=False).agg({"dismissal_kind": sum, "match_id": lambda x: x.nunique()})
dismissalInfo = dismissalInfo.rename(index=str, columns={"dismissal_kind": "total_wickets", "match_id": "matches_played"})

consistency = pd.merge(consistency,dismissalInfo, on=["bowler"])

consistency = consistency[consistency['matches_played'] > 10]
consistency = consistency.sort_values('consec_matches', ascending=False)
consistency

,bowler,consec_matches,total_wickets,matches_played
80,DJ Bravo,24,122,103
229,R Vinay Kumar,15,101,101
169,M Morkel,14,77,70
6,A Nehra,13,98,82
58,BW Hilfenhaus,12,22,17
276,SL Malinga,12,143,98
215,PP Ojha,11,89,90
17,AB Dinda,10,68,72
235,RJ Harris,10,45,37
115,Imran Tahir,10,29,20


In [9]:
# Highest player contribution (Player Wickets / Total Team Wickets)
# Atleast should have played 10 matches

bowlerContributionInfo = ballInfo[['match_id','inning','bowler','dismissal_kind']]
bowlerContributionInfo['dismissal_kind'] = bowlerContributionInfo['dismissal_kind'].fillna(0)

teamWktsInfo = bowlerContributionInfo
teamWktsInfo['dismissal_kind'] = teamWktsInfo['dismissal_kind'].replace(['caught', 'bowled', 'run out', 'lbw', 'retired hurt', 'stumped','caught and bowled', 'hit wicket', 'obstructing the field'],1)
teamWkts = teamWktsInfo.groupby(['match_id','inning'],as_index=False)['dismissal_kind'].sum()

playerWktsInfo = bowlerContributionInfo
playerWktsInfo['dismissal_kind'] = playerWktsInfo['dismissal_kind'].replace(['caught', 'bowled', 'lbw', 'stumped','caught and bowled', 'hit wicket'],1)
playerWktsInfo['dismissal_kind'] = playerWktsInfo['dismissal_kind'].replace(['run out', 'retired hurt', 'hit wicket'],0)
playerWkts = playerWktsInfo.groupby(['match_id','inning','bowler'],as_index=False)['dismissal_kind'].sum()

teamWkts = teamWkts.rename(index=str, columns={"dismissal_kind": "team_wickets"})
playerWkts = playerWkts.rename(index=str, columns={"dismissal_kind": "player_wickets"})
contribution = pd.merge(teamWkts,playerWkts,on=['match_id','inning'])

def bowlerCntr(x):
    if x["team_wickets"] == 0:
        return 0.0
    else:
        return x["player_wickets"]/x["team_wickets"]

contribution["contribution"] = contribution.apply(bowlerCntr, axis=1)
contributionInfo = contribution.groupby(['bowler'],as_index=False)['contribution'].mean()

notbowlerDismissals = ['run out','hit wicket','obstructing the field','hit wicket','retired hurt']
dismissalInfo = ballInfo[['match_id','bowler','dismissal_kind']]
dismissalInfo = dismissalInfo[~dismissalInfo['dismissal_kind'].isin(notbowlerDismissals)]
dismissalInfo = dismissalInfo.fillna(0)
dismissalInfo['dismissal_kind'] = dismissalInfo['dismissal_kind'].replace(['caught', 'bowled', 'lbw', 'stumped','caught and bowled'], 1)
dismissalInfo = dismissalInfo.groupby(['bowler'],as_index=False).agg({"dismissal_kind": sum, "match_id": lambda x: x.nunique()})
dismissalInfo = dismissalInfo.rename(index=str, columns={"dismissal_kind": "total_wickets", "match_id": "matches_played"})

contributionInfo = pd.merge(contributionInfo,dismissalInfo, on=["bowler"])

contributionInfo = contributionInfo.sort_values('contribution',ascending=False)
contributionInfo = contributionInfo[contributionInfo['matches_played'] > 10]
contributionInfo

/Users/suribabu/Projects/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/suribabu/Projects/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/suribabu/Projects/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

,bowler,contribution,total_wickets,matches_played
224,R Rampaul,0.289621,16,12
301,Sohail Tanvir,0.266667,22,11
276,SL Malinga,0.265602,143,98
325,WPUJC Vaas,0.261142,18,13
154,KW Richardson,0.256746,18,14
79,DE Bollinger,0.256526,37,27
115,Imran Tahir,0.256012,29,20
202,P Awana,0.251431,39,33
243,RS Bopara,0.244728,11,14
324,WD Parnell,0.242918,26,26


In [10]:
# If IPL is played as league, with no knockout matches and based on points after playing with all temas (home & away) like BPL, LaLiga
# Which teams would have won IPL

seasonInfo = matchInfo
seasonInfo["matchNum"] = 0
seasonInfo.loc[0,'matchNum'] = 1;
for i in range(1,len(seasonInfo)):
    if seasonInfo.loc[i,'season'] == seasonInfo.loc[i-1,'season']:
        seasonInfo.loc[i,'matchNum'] = seasonInfo.loc[i-1,'matchNum'] + 1
    else:
        seasonInfo.loc[i,'matchNum'] = 1
nonplayOff = {
    2008: 56,
    2009: 56,
    2010: 56,
    2011: 70,
    2012: 72,
    2013: 72,
    2014: 56,
    2015: 56,
    2016: 56,
}
seasonInfo["isPlayOff"] = False
def isPlayOff(x):
    if x['matchNum'] <= nonplayOff[x.season]:
        return False
    else:
        return True
        
seasonInfo["isPlayOff"] = seasonInfo.apply(isPlayOff,axis=1)
seasonInfo = seasonInfo[seasonInfo.isPlayOff == False]

seasonMatches = seasonInfo
seasonMatches.drop(['id'],axis=1,inplace=True)
seasonInfo.reset_index(drop=True)

pointsTable = pd.DataFrame(columns=['season','team','points'])

start=1
for index, row in seasonInfo.iterrows():
    pointsTable.loc[start, 'season'] = row['season']
    if pd.isnull(row['winner']):
        pointsTable.loc[start, 'team'] = row['winner']
        pointsTable.loc[start, 'points'] = 2
        start = start + 1
    else:
        pointsTable.loc[start, 'team'] = row['team1']
        pointsTable.loc[start, 'points'] = 1
        start = start + 1
        pointsTable.loc[start, 'season'] = row['season']
        pointsTable.loc[start, 'team'] = row['team2']
        pointsTable.loc[start, 'points'] = 1

pointsTable = pointsTable.groupby(['season','team'],as_index=False)['points'].sum()

pointsTable.sort_values(['season','points'], ascending=[1,0]).groupby('season').head(1)

/Users/suribabu/Projects/Anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,season,team,points
4,2008,Kolkata Knight Riders,9
8,2009,Chennai Super Kings,10
21,2010,Mumbai Indians,10
24,2011,Chennai Super Kings,11
35,2012,Deccan Chargers,11
50,2013,Royal Challengers Bangalore,11
59,2014,Sunrisers Hyderabad,10
60,2015,Chennai Super Kings,11
74,2016,Royal Challengers Bangalore,11


In [11]:
# Highest unbeaten/winless streak of all teams

matchResult = pd.DataFrame(columns=['season','team','result'])
start = 0
for i in range(0,len(matchInfo)):
    if pd.isnull(matchInfo.loc[i,"winner"]):
        matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
        matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
        matchResult.loc[start,"team"] = matchInfo.loc[i,"team1"]
        matchResult.loc[start,'result'] = 'draw'
        start = start + 1
        matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
        matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
        matchResult.loc[start,"team"] = matchInfo.loc[i,"team2"]
        matchResult.loc[start,'result'] = 'draw'
    else:
        if matchInfo.loc[i, "winner"] == matchInfo.loc[i, "team1"]:
            matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
            matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
            matchResult.loc[start,"team"] = matchInfo.loc[i,"team1"]
            matchResult.loc[start,'result'] = 'win'
            start = start + 1
            matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
            matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
            matchResult.loc[start,"team"] = matchInfo.loc[i,"team2"]
            matchResult.loc[start,'result'] = 'loss'
        else:
            matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
            matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
            matchResult.loc[start,"team"] = matchInfo.loc[i,"team2"]
            matchResult.loc[start,'result'] = 'win'
            start = start + 1
            matchResult.loc[start,"season"] = matchInfo.loc[i,"season"]
            matchResult.loc[start,"match_id"] = matchInfo.loc[i,"id"]
            matchResult.loc[start,"team"] = matchInfo.loc[i,"team2"]
            matchResult.loc[start,'result'] = 'loss'
    start = start + 1

matchResult = matchResult.sort_values(['match_id'], ascending=[1])
teamList = list(matchResult['team'].unique())
resultInfo = []
tempResult = []

for team in teamList:
    resultInfo.append({
            'team': team,
            'unbeaten': 0,
            'winless': 0
        })
    tempResult.append({
            'team': team,
            'prevResult': None,
            'count': 0
        })

for i in range(0,len(matchResult)):
    teamIdx = -1
    for idx in range(len(tempResult)):
        if tempResult[idx]["team"] == matchResult.loc[i,'team']:
            teamIdx = idx
            break
    if tempResult[teamIdx]["prevResult"] == None:
        tempResult[teamIdx]["prevResult"] = matchResult.loc[i,'result']
        tempResult[teamIdx]["count"] = 1
    else:
        if matchResult.loc[i,'result'] != tempResult[teamIdx]["prevResult"]:
            if matchResult.loc[i,'result'] == "draw":
                tempResult[teamIdx]["count"] = tempResult[teamIdx]["count"] + 1
            elif tempResult[teamIdx]["prevResult"] == "draw":
                tempResult[teamIdx]["prevResult"] = matchResult.loc[i,'result']
                tempResult[teamIdx]["count"] = tempResult[teamIdx]["count"] + 1
            else:
                tempResult[teamIdx]["prevResult"] = matchResult.loc[i,'result']
                tempResult[teamIdx]["count"] = 1
        else:
            tempResult[teamIdx]["count"] = tempResult[teamIdx]["count"] + 1
            
    for idx in range(len(resultInfo)):
        if tempResult[teamIdx]["team"] == resultInfo[idx]['team']:
            if tempResult[teamIdx]["prevResult"] == 'win':
                if tempResult[teamIdx]["count"] > resultInfo[idx]["unbeaten"]:
                    resultInfo[idx]["unbeaten"] = tempResult[teamIdx]["count"]
            elif tempResult[teamIdx]["prevResult"] == 'loss':
                if tempResult[teamIdx]["count"] > resultInfo[idx]["winless"]:
                    resultInfo[idx]["winless"] = tempResult[teamIdx]["count"]
            elif tempResult[teamIdx]["prevResult"] == 'draw':
                if tempResult[teamIdx]["count"] > resultInfo[idx]["unbeaten"]:
                    resultInfo[idx]["unbeaten"] = tempResult[teamIdx]["count"]
                if tempResult[teamIdx]["count"] > resultInfo[idx]["winless"]:
                    resultInfo[idx]["winless"] = tempResult[teamIdx]["count"]
            break
            
resultInfo

[{'team': 'Kolkata Knight Riders', 'unbeaten': 4, 'winless': 4},
 {'team': 'Royal Challengers Bangalore', 'unbeaten': 3, 'winless': 3},
 {'team': 'Chennai Super Kings', 'unbeaten': 5, 'winless': 4},
 {'team': 'Kings XI Punjab', 'unbeaten': 3, 'winless': 6},
 {'team': 'Delhi Daredevils', 'unbeaten': 4, 'winless': 6},
 {'team': 'Rajasthan Royals', 'unbeaten': 4, 'winless': 3},
 {'team': 'Mumbai Indians', 'unbeaten': 5, 'winless': 3},
 {'team': 'Deccan Chargers', 'unbeaten': 3, 'winless': 5},
 {'team': 'Pune Warriors', 'unbeaten': 3, 'winless': 8},
 {'team': 'Kochi Tuskers Kerala', 'unbeaten': 2, 'winless': 2},
 {'team': 'Sunrisers Hyderabad', 'unbeaten': 4, 'winless': 2},
 {'team': 'Rising Pune Supergiants', 'unbeaten': 1, 'winless': 2},
 {'team': 'Gujarat Lions', 'unbeaten': 2, 'winless': 2}]

In [14]:
resultInfoStr = '[{"team":"Kolkata Knight Riders","unbeaten":4,"winless":4},{"team":"Royal Challengers Bangalore","unbeaten":3,"winless":3},{"team":"Chennai Super Kings","unbeaten":5,"winless":4},{"team":"Kings XI Punjab","unbeaten":3,"winless":6},{"team":"Delhi Daredevils","unbeaten":4,"winless":6},{"team":"Rajasthan Royals","unbeaten":4,"winless":3},{"team":"Mumbai Indians","unbeaten":5,"winless":3},{"team":"Deccan Chargers","unbeaten":3,"winless":5},{"team":"Pune Warriors","unbeaten":3,"winless":8},{"team":"Kochi Tuskers Kerala","unbeaten":2,"winless":2},{"team":"Sunrisers Hyderabad","unbeaten":4,"winless":2},{"team":"Rising Pune Supergiants","unbeaten":1,"winless":2},{"team":"Gujarat Lions","unbeaten":2,"winless":2}]'
pd.read_json(str(resultInfoStr))

,team,unbeaten,winless
0,Kolkata Knight Riders,4,4
1,Royal Challengers Bangalore,3,3
2,Chennai Super Kings,5,4
3,Kings XI Punjab,3,6
4,Delhi Daredevils,4,6
5,Rajasthan Royals,4,3
6,Mumbai Indians,5,3
7,Deccan Chargers,3,5
8,Pune Warriors,3,8
9,Kochi Tuskers Kerala,2,2
